# MIMIC III Demo: Creation, connection and model

This is a notebook to build incrementally the connection and the model to the MIMIC III database.

## Database creation

The tables have been created using the script provded in the <a href="https://github.com/MIT-LCP/mimic-code/blob/master/buildmimic/postgres/postgres_create_tables.sql">Git-hub official repository</a>. The tables ID are defined as INTEGER, not as SERIAL, so I have to manual provide a primary key using the `unrestricted_primary_key` instruction.

In [ ]:
require 'pg'
conn = PG.connect(dbname: 'postgres', host: 'postgres', user: 'postgres', password: 'password')

In [ ]:
conn.exec('CREATE DATABASE mimic3_demo')

## Tables creation

In [ ]:
conn = PG.connect(dbname: 'mimic3_demo', host: 'postgres', user: 'postgres', password: 'password')
conn.exec('CREATE SCHEMA mimiciii')
conn.exec('SET search_path TO mimiciii')

input_file = File.open('/mimic3/1.4/create_tables.sql')
conn.exec(input_file.read)

## Sequel connection

Connection is made through the use of the **Sequel** library. 

In [1]:
require 'sequel'
DB_MIMIC = Sequel.connect(adapter: 'postgres',
                          host: 'postgres',
                          user: 'postgres',
                          password: 'password',
                          database: 'mimic3_demo',
                          search_path: ['mimiciii']
                          )

#<Sequel::Postgres::Database: {:adapter=>"postgres", :host=>"postgres", :user=>"postgres", :password=>"password", :database=>"mimic3_demo", :search_path=>["mimiciii"]}>

## ER Schema

I found on the Git-Hub repository of MIMIC this image representing the ER schema

<img src="img/ER-Schema.png"></img>


## Model

Here is the definitions for all the classes dealing with ORM.

#### Patients

In [2]:
Object.send(:remove_const, :Patient) if defined? (Patient)

class Patient < Sequel::Model(DB_MIMIC)  # It search for 'patients' table
  unrestrict_primary_key
  
  one_to_many :admissions, key: :subject_id, primary_key: :subject_id, class: :Admission
  one_to_many :drgcodes, key: :subject_id, primary_key: :subject_id, class: :DrgCode
  one_to_many :microbiologyevents, key: :subject_id, primary_key: :subject_id, class: :MicrobiologyEvent
  one_to_many :callout, key: :subject_id, primary_key: :subject_id, class: :Callout
  one_to_many :diagnoses_icd, key: :subject_id, primary_key: :subject_id, class: :DiagnoseIcd
  one_to_many :icustays, key: :subject_id, primary_key: :subject_id, class: :Icustay
  one_to_many :datetimeevents, key: :subject_id, primary_key: :subject_id, class: :DateTimeEvent
  one_to_many :cptevents, key: :subject_id, primary_key: :subject_id, class: :CptEvent
  one_to_many :prescriptions, key: :subject_id, primary_key: :subject_id, class: :Prescription
  one_to_many :chartevents, key: :subject_id, primary_key: :subject_id, class: :ChartEvent
  one_to_many :transfers, key: :subject_id, primary_key: :subject_id, class: :Transfer
  one_to_many :inputevents_cv, key: :subject_id, primary_key: :subject_id, class: :InputEventCV
  one_to_many :noteevents, key: :subject_id, primary_key: :subject_id, class: :NoteEvent
  one_to_many :services, key: :subject_id, primary_key: :subject_id, class: :Service
  one_to_many :procedures_icd, key: :subject_id, primary_key: :subject_id, class: :ProcedureIcd
  one_to_many :outputevents, key: :subject_id, primary_key: :subject_id, class: :OutputEvent
  one_to_many :procedureevents_mv, key: :subject_id, primary_key: :subject_id, class: :ProcedureEventMV
  one_to_many :inputevents_mv, key: :subject_id, primary_key: :subject_id, class: :InputEventMV
end
nil

#### Admissions

In [3]:
Object.send(:remove_const, :Admission) if defined? (Admission)

class Admission < Sequel::Model(DB_MIMIC)
  unrestrict_primary_key
  
  many_to_one :patient, key: :subject_id, primary_key: :subject_id, class: :Patient
  
  one_to_many :drgcodes, key: :hadm_id, primary_key: :hadm_id, class: :DrgCode
  one_to_many :microbiologyevents, key: :hadm_id, primary_key: :hadm_id, class: :MicrobiologyEvent
  one_to_many :callout, key: :hadm_id, primary_key: :hadm_id, class: :Callout
  one_to_many :diagnoses_icd, key: :hadm_id, primary_key: :hadm_id, class: :DiagnoseIcd
  one_to_many :icustays, key: :hadm_id, primary_key: :hadm_id, class: :Icustay
  one_to_many :datetimeevents, key: :hadm_id, primary_key: :hadm_id, class: :DateTimeEvent
  one_to_many :cptevents, key: :hadm_id, primary_key: :hadm_id, class: :CptEvent
  one_to_many :prescriptions, key: :hadm_id, primary_key: :hadm_id, class: :Prescription
  one_to_many :chartevents, key: :hadm_id, primary_key: :hadm_id, class: :ChartEvent
  one_to_many :transfers, key: :hadm_id, primary_key: :hadm_id, class: :Transfer
  one_to_many :inputevents_cv, key: :hadm_id, primary_key: :hadm_id, class: :InputEventCV
  one_to_many :noteevents, key: :hadm_id, primary_key: :hadm_id, class: :NoteEvent
  one_to_many :services, key: :hadm_id, primary_key: :hadm_id, class: :Service
  one_to_many :procedures_icd, key: :hadm_id, primary_key: :hadm_id, class: :ProcedureIcd
  one_to_many :outputevents, key: :hadm_id, primary_key: :hadm_id, class: :OutputEvent
  one_to_many :procedureevents_mv, key: :hadm_id, primary_key: :hadm_id, class: :ProcedureEventMV
  one_to_many :inputevents_mv, key: :hadm_id, primary_key: :hadm_id, class: :InputEventMV
end
nil

#### DRG codes

In [ ]:
Object.send(:remove_const, :DrgCode) if defined? (DrgCode)

class DrgCode < Sequel::Model(DB_MIMIC[:drgcodes])  # search for 'drgcodes' table
  unrestrict_primary_key
  
  many_to_one :patient, key: :subject_id, primary_key: :subject_id, class: :Patient
  many_to_one :admission, key: :hadm_id, primary_key: :hadm_id, class: :Admission
end
nil

#### Microbiologyevents

In [ ]:
Object.send(:remove_const, :MicrobiologyEvent) if defined? (MicrobiologyEvent)

class MicrobiologyEvent < Sequel::Model(DB_MIMIC[:microbiologyevents])
  unrestrict_primary_key
  
  many_to_one :patient, key: :subject_id, primary_key: :subject_id, class: :Patient
  many_to_one :admission, key: :hadm_id, primary_key: :hadm_id, class: :Admission
end
nil

#### Callout

In [ ]:
Object.send(:remove_const, :Callout) if defined? (Callout)

class Callout < Sequel::Model(DB_MIMIC[:callout])
  unrestrict_primary_key
  
  many_to_one :patient, key: :subject_id, primary_key: :subject_id, class: :Patient
  many_to_one :admission, key: :hadm_id, primary_key: :hadm_id, class: :Admission
end
nil

#### Diagnoses_icd

In [ ]:
Object.send(:remove_const, :DiagnoseIcd) if defined? (DiagnoseIcd)

class DiagnoseIcd < Sequel::Model(DB_MIMIC[:diagnoses_icd])
  unrestrict_primary_key
  
  many_to_one :patient, key: :subject_id, primary_key: :subject_id, class: :Patient
  many_to_one :admission, key: :hadm_id, primary_key: :hadm_id, class: :Admission
end
nil

#### CPT events

In [ ]:
Object.send(:remove_const, :CptEvent) if defined? (CptEvent)

class CptEvent < Sequel::Model(DB_MIMIC[:cptevents])
  unrestrict_primary_key
  
  many_to_one :patient, key: :subject_id, primary_key: :subject_id, class: :Patient
  many_to_one :admission, key: :hadm_id, primary_key: :hadm_id, class: :Admission
end
nil

#### Services

In [ ]:
Object.send(:remove_const, :Service) if defined? (Service)

class Service < Sequel::Model(DB_MIMIC)
  unrestrict_primary_key
  
  many_to_one :patient, key: :subject_id, primary_key: :subject_id, class: :Patient
  many_to_one :admission, key: :hadm_id, primary_key: :hadm_id, class: :Admission
end
nil

#### Procedures ICD

In [ ]:
Object.send(:remove_const, :ProcedureIcd) if defined? (ProcedureIcd)

class ProcedureIcd < Sequel::Model(DB_MIMIC[:procedures_icd])
  unrestrict_primary_key
  
  many_to_one :patient, key: :subject_id, primary_key: :subject_id, class: :Patient
  many_to_one :admission, key: :hadm_id, primary_key: :hadm_id, class: :Admission
end
nil

#### Icustays

In [ ]:
Object.send(:remove_const, :Icustay) if defined? (Icustay)

class Icustay < Sequel::Model(DB_MIMIC)
  unrestrict_primary_key
  
  many_to_one :patient, key: :subject_id, primary_key: :subject_id, class: :Patient
  many_to_one :admission, key: :hadm_id, primary_key: :hadm_id, class: :Admission
  
  one_to_many :prescriptions, key: :icustay_id, primary_key: :icustay_id, class: :Icustay
  one_to_many :datetimeevents, key: :icustay_id, primary_key: :icustay_id, class: :DateTimeEvent
  one_to_many :chartevents, key: :icustay_id, primary_key: :icustay_id, class: :ChartEvent
  one_to_many :transfers, key: :icustay_id, primary_key: :icustay_id, class: :Transfer
  one_to_many :inputevents_cv, key: :icustay_id, primary_key: :icustay_id, class: :InputEventCV
  one_to_many :outputevents, key: :icustay_id, primary_key: :icustay_id, class: :OutputEvent
  one_to_many :procedureevents_mv, key: :icustay_id, primary_key: :icustay_id, class: :ProcedureEventMV
  one_to_many :inputevents_mv, key: :icustay_id, primary_key: :icustay_id, class: :InputEventMV
end
nil

#### Prescriptions

In [ ]:
Object.send(:remove_const, :Prescription) if defined? (Prescription)

class Prescription < Sequel::Model(DB_MIMIC)
  unrestrict_primary_key
  
  many_to_one :patient, key: :subject_id, primary_key: :subject_id, class: :Patient
  many_to_one :admission, key: :hadm_id, primary_key: :hadm_id, class: :Admission
  many_to_one :icustay, key: :icustay_id, primary_key: :icustay_id, class: :Icustay
end
nil

#### Transfers

In [ ]:
Object.send(:remove_const, :Transfer) if defined? (Transfer)

class Transfer < Sequel::Model(DB_MIMIC)
  unrestrict_primary_key
  
  many_to_one :patient, key: :subject_id, primary_key: :subject_id, class: :Patient
  many_to_one :admission, key: :hadm_id, primary_key: :hadm_id, class: :Admission
  many_to_one :icustay, key: :icustay_id, primary_key: :icustay_id, class: :Icustay
end
nil

#### Caregivers

In [ ]:
Object.send(:remove_const, :Caregiver) if defined? (Caregiver)

class Caregiver < Sequel::Model(DB_MIMIC)
  unrestrict_primary_key
  
  one_to_many :datetimeevent, key: :cgid, primary_key: :cgid
  one_to_many :chartevent, key: :cgid, primary_key: :cgid
  one_to_many :inputevents_cv, key: :cgid, primary_key: :cgid
  one_to_many :noteevents, key: :cgid, primary_key: :cgid
  one_to_many :outputevents, key: :cgid, primary_key: :cgid
  one_to_many :procedureevents_mv, key: :cgid, primary_key: :cgid
  one_to_many :inputevents_mv, key: :cgid, primary_key: :cgid
end
nil

#### Input events cv

In [ ]:
Object.send(:remove_const, :InputEventCV) if defined? (InputEventCV)

class InputEventCV < Sequel::Model(DB_MIMIC[:inputevents_cv])
  unrestrict_primary_key
  
  many_to_one :patient, key: :subject_id, primary_key: :subject_id, class: :Patient
  many_to_one :admission, key: :hadm_id, primary_key: :hadm_id, class: :Admission
  many_to_one :icustay, key: :icustay_id, primary_key: :icustay_id, class: :Icustay
  many_to_one :caregiver, key: :cgid, primary_key: :cgid, class: :Caregiver
end
nil

#### Output events

In [ ]:
Object.send(:remove_const, :OutputEvent) if defined? (OutputEvent)

class OutputEvent < Sequel::Model(DB_MIMIC[:outputevents])
  unrestrict_primary_key
  
  many_to_one :patient, key: :subject_id, primary_key: :subject_id, class: :Patient
  many_to_one :admission, key: :hadm_id, primary_key: :hadm_id, class: :Admission
  many_to_one :icustay, key: :icustay_id, primary_key: :icustay_id, class: :Icustay
  many_to_one :caregiver, key: :cgid, primary_key: :cgid, class: :Caregiver
end
nil

#### Procedure events mv

In [ ]:
Object.send(:remove_const, :ProcedureEventMV) if defined? (ProcedureEventMV)

class ProcedureEventMV < Sequel::Model(DB_MIMIC[:procedureevents_mv])
  unrestrict_primary_key
  
  many_to_one :patient, key: :subject_id, primary_key: :subject_id, class: :Patient
  many_to_one :admission, key: :hadm_id, primary_key: :hadm_id, class: :Admission
  many_to_one :icustay, key: :icustay_id, primary_key: :icustay_id, class: :Icustay
  many_to_one :caregiver, key: :cgid, primary_key: :cgid, class: :Caregiver
end
nil

#### Input events mv

In [ ]:
Object.send(:remove_const, :InputEventMV) if defined? (InputEventMV)

class InputEventMV < Sequel::Model(DB_MIMIC[:inputevents_mv])
  unrestrict_primary_key
  
  many_to_one :patient, key: :subject_id, primary_key: :subject_id, class: :Patient
  many_to_one :admission, key: :hadm_id, primary_key: :hadm_id, class: :Admission
  many_to_one :icustay, key: :icustay_id, primary_key: :icustay_id, class: :Icustay
  many_to_one :caregiver, key: :cgid, primary_key: :cgid, class: :Caregiver
end
nil

#### Note events

In [ ]:
Object.send(:remove_const, :NoteEvent) if defined? (NoteEvent)

class NoteEvent < Sequel::Model(DB_MIMIC[:noteevents])
  unrestrict_primary_key
  
  many_to_one :patient, key: :subject_id, primary_key: :subject_id, class: :Patient
  many_to_one :admission, key: :hadm_id, primary_key: :hadm_id, class: :Admission
  many_to_one :caregiver, key: :cgid, primary_key: :cgid, class: :Caregiver
end
nil

#### Definitions: Items

In [ ]:
Object.send(:remove_const, :DItem) if defined? (DItem)

class DItem < Sequel::Model(DB_MIMIC)
  unrestrict_primary_key
  
  one_to_many :datetimeevents, key: :itemid, primary_key: :itemid, class: :DateTimeEvent
  one_to_many :chartevents, key: :itemid, primary_key: :itemid, class: :ChartEvent
end
nil

#### Chart events

In [ ]:
Object.send(:remove_const, :ChartEvent) if defined? (ChartEvent)

class ChartEvent < Sequel::Model(DB_MIMIC[:chartevents])
  unrestrict_primary_key
  
  many_to_one :patient, key: :subject_id, primary_key: :subject_id, class: :Patient
  many_to_one :admission, key: :hadm_id, primary_key: :hadm_id, class: :Admission
  many_to_one :icustay, key: :icustay_id, primary_key: :icustay_id, class: :Icustay
  many_to_one :d_item, key: :itemid, primary_key: :itemid, class: :DItem
  many_to_one :caregiver, key: :cgid, primary_key: :cgid, class: :Caregiver
end
nil

#### Date-Time events

In [ ]:
Object.send(:remove_const, :DateTimeEvent) if defined? (DateTimeEvent)

class DateTimeEvent < Sequel::Model(DB_MIMIC[:datetimeevents])
  unrestrict_primary_key
  
  many_to_one :patients, key: :subject_id, primary_key: :subject_id, class: :Patient
  many_to_one :admissions, key: :hadm_id, primary_key: :hadm_id, class: :Admission
  many_to_one :icustays, key: :icustay_id, primary_key: :icustay_id, class: :Icustay
  many_to_one :d_items, key: :itemid, primary_key: :itemid, class: :DItem
  many_to_one :caregivers, key: :cgid, primary_key: :cgid, class: :Caregiver
end
nil

#### Lab items description

In [ ]:
Object.send(:remove_const, :DLabItem) if defined? (DLabItem)

class DLabItem < Sequel::Model(DB_MIMIC[:d_labitems])
  unrestrict_primary_key
  
  one_to_many :labevents, key: :itemid, primary_key: :itemid, class: :LabEvent
end
nil

#### Lab events

In [ ]:
Object.send(:remove_const, :LabEvent) if defined? (LabEvent)

class LabEvent < Sequel::Model(DB_MIMIC[:labevents])
  unrestrict_primary_key
  
  many_to_one :patient, key: :subject_id, primary_key: :subject_id, class: :Patient
  many_to_one :admission, key: :hadm_id, primary_key: :hadm_id, class: :Admission
  many_to_one :d_labitem, key: :itemid, primary_key: :itemid, class: :DLabItem
end
nil

#### Diagnoses icd description

In [ ]:
Object.send(:remove_const, :DIcdDiagnose) if defined? (DIcdDiagnose)

class DIcdDiagnose < Sequel::Model(DB_MIMIC)
  unrestrict_primary_key
end
nil

#### Cpt descriptions

In [ ]:
Object.send(:remove_const, :DCpt) if defined? (DCpt)

class DCpt < Sequel::Model(DB_MIMIC[:d_cpt])
  unrestrict_primary_key
end
nil

#### Icd procedures description

In [ ]:
Object.send(:remove_const, :DIcdProcedure) if defined? (DIcdProcedure)

class DIcdProcedure < Sequel::Model(DB_MIMIC)
  unrestrict_primary_key
end
nil

# Test

In [ ]:
Patient.create(row_id: 1, subject_id: 8888, gender: 'M', dob: Date.parse('196-10-12'), expire_flag: '0')
puts Patient.first.methods.grep(/add/)
Patient.first.delete
nil

In [ ]:
Patient.first.add_admission(row_id: 1, subject_id: 8888, hadm_id: 9999, 
                            admittime: Date.today, dischtime: Date.today, admission_type: 'EMERGENCY',
                            admission_location: 'Here', discharge_location: 'Here', insurance: 'None',
                            ethnicity: 'boh', has_chartevents_data: 8)

In [ ]:
Patient.first.methods.grep(/add/)

In [ ]:
Admission.first.methods.grep(/add/)

In [ ]:
DItem.create(row_id: 1, itemid: 1, label: 'Strope')

In [ ]:
Patient.first.delete

In [13]:
Admission.first.patient

#<Patient @values={:row_id=>9467, :subject_id=>10006, :gender=>"F", :dob=>2094-03-05 00:00:00 +0000, :dod=>2165-08-12 00:00:00 +0000, :dod_hosp=>2165-08-12 00:00:00 +0000, :dod_ssn=>2165-08-12 00:00:00 +0000, :expire_flag=>1}>